# Dollar General Scrape

This notebook is part 1 of a project to generate a county-level map of all the DG stores in the US at the time the notebook is run. It works by first scraping Dollar General's website to find the number of stores in each town, and then maps that town with the associated County Code by the Census Bureau. You will need to install Pandas and Icecream to run this notebook. 

Since DG has tens of thousands of stores, it is not possible to list all of the towns manually. Instead, We're going to scrape the state-level directory for a list of towns with a DG, and then scrape the town-level directories for a list of stores in that town. 

In [5]:
from bs4 import BeautifulSoup
import pandas as pd
import requests ## a library that returns information from websites.
from icecream import ic
from random import randrange
import time

In [120]:
pip install icecream

Note: you may need to restart the kernel to use updated packages.


# Part 1
The first step is to scrape a list of URLs to Dollar General's state-level directories. The top-level directory is at the following link:

In [6]:
base_url="https://www.dollargeneral.com/store-directory/"

In [7]:
response=requests.get(base_url)
response.status_code

200

In [33]:
def snoozer():
    snooze=randrange(1)
    time.sleep(snooze)

In [9]:
soup=BeautifulSoup(response.text, "html.parser")

In [10]:
soup


<!DOCTYPE HTML>

<html lang="en">
<head>
<meta charset="utf-8"/>
<title>
        Store Directory | Find Dollar General Stores Near You
    </title>
<meta content="Find the nearest Dollar General store in your area for everyday low prices on household essentials, groceries, and more. Shop online or in-store today!" name="description"/>
<!-- <meta data-sly-test="false" name="robots" content="noindex,nofollow"/> -->
<meta content="store-locations" name="template"/>
<meta content="width=device-width, initial-scale=1" name="viewport"/>
<base href="https://www.dollargeneral.com"/>
<link href="https://fonts.googleapis.com/css2?family=Montserrat:ital,wght@0,100;0,200;0,300;0,400;0,500;0,600;0,700;0,800;0,900;1,100;1,200;1,300;1,400;1,500;1,600;1,700;1,800;1,900&amp;display=swap" rel="stylesheet"/>
<script src="https://polyfill.io/v3/polyfill.min.js?features=HTMLTemplateElement%2CString.prototype.includes%2CArray.from%2CNodeList.prototype.forEach%2CURLSearchParams%2CElement.prototype.append">


In [11]:
States=soup.find_all("div", class_="state-list-item")
States

[<div class="state-list-item"><p class="location-list-item"><a href="/content/dollargeneral/en/store-directory/al.html" target="_self" title="Alabama">Alabama</a></p></div>,
 <div class="state-list-item"><p class="location-list-item"><a href="/content/dollargeneral/en/store-directory/az.html" target="_self" title="Arizona">Arizona</a></p></div>,
 <div class="state-list-item"><p class="location-list-item"><a href="/content/dollargeneral/en/store-directory/ar.html" target="_self" title="Arkansas">Arkansas</a></p></div>,
 <div class="state-list-item"><p class="location-list-item"><a href="/content/dollargeneral/en/store-directory/ca.html" target="_self" title="California">California</a></p></div>,
 <div class="state-list-item"><p class="location-list-item"><a href="/content/dollargeneral/en/store-directory/co.html" target="_self" title="Colorado">Colorado</a></p></div>,
 <div class="state-list-item"><p class="location-list-item"><a href="/content/dollargeneral/en/store-directory/ct.html" 

In [12]:
len(States)

50

In [13]:
print(States[0].get('href'))

None


In [14]:
states=[state.a for state in States]
links=[state.get('href') for state in states]
links
#Here are the links to state-level directories.    

['/content/dollargeneral/en/store-directory/al.html',
 '/content/dollargeneral/en/store-directory/az.html',
 '/content/dollargeneral/en/store-directory/ar.html',
 '/content/dollargeneral/en/store-directory/ca.html',
 '/content/dollargeneral/en/store-directory/co.html',
 '/content/dollargeneral/en/store-directory/ct.html',
 '/content/dollargeneral/en/store-directory/de.html',
 '/content/dollargeneral/en/store-directory/dc.html',
 '/content/dollargeneral/en/store-directory/fl.html',
 '/content/dollargeneral/en/store-directory/ga.html',
 '/content/dollargeneral/en/store-directory/id.html',
 '/content/dollargeneral/en/store-directory/il.html',
 '/content/dollargeneral/en/store-directory/in.html',
 '/content/dollargeneral/en/store-directory/ia.html',
 '/content/dollargeneral/en/store-directory/ks.html',
 '/content/dollargeneral/en/store-directory/ky.html',
 '/content/dollargeneral/en/store-directory/la.html',
 '/content/dollargeneral/en/store-directory/me.html',
 '/content/dollargeneral/en/

In [164]:
states=[state.a for state in States]
links=[state.get('title') for state in states]
links
    

['Alabama',
 'Arizona',
 'Arkansas',
 'California',
 'Colorado',
 'Connecticut',
 'Delaware',
 'District of Columbia',
 'Florida',
 'Georgia',
 'Idaho',
 'Illinois',
 'Indiana',
 'Iowa',
 'Kansas',
 'Kentucky',
 'Louisiana',
 'Maine',
 'Maryland',
 'Massachusetts',
 'Michigan',
 'Minnesota',
 'Mississippi',
 'Missouri',
 'Montana',
 'Nebraska',
 'Nevada',
 'New Hampshire',
 'New Jersey',
 'New Mexico',
 'New York',
 'North Carolina',
 'North Dakota',
 'Ohio',
 'Oklahoma',
 'Oregon',
 'Pennsylvania',
 'Rhode Island',
 'South Carolina',
 'South Dakota',
 'Tennessee',
 'Texas',
 'Utah',
 'Vermont',
 'Virgina',
 'Virginia',
 'Washington',
 'West Virginia',
 'Wisconsin',
 'Wyoming']

DG uses lower-case abbreviations for each state in its URLs. The following code creates a dictionary mapping each state name to its abbreviation.

In [15]:
state_list=[]
state_links=[]
for state in states:
    link=state.get('href')
    newlink=link.replace("/content/dollargeneral/en/store-directory/","").replace(".html","")
    state_name=state.get('title')
    dict={"State Name":state_name, "state_ID":newlink}
    state_list.append(dict)
    state_links.append(newlink)
state_list
    

[{'State Name': 'Alabama', 'state_ID': 'al'},
 {'State Name': 'Arizona', 'state_ID': 'az'},
 {'State Name': 'Arkansas', 'state_ID': 'ar'},
 {'State Name': 'California', 'state_ID': 'ca'},
 {'State Name': 'Colorado', 'state_ID': 'co'},
 {'State Name': 'Connecticut', 'state_ID': 'ct'},
 {'State Name': 'Delaware', 'state_ID': 'de'},
 {'State Name': 'District of Columbia', 'state_ID': 'dc'},
 {'State Name': 'Florida', 'state_ID': 'fl'},
 {'State Name': 'Georgia', 'state_ID': 'ga'},
 {'State Name': 'Idaho', 'state_ID': 'id'},
 {'State Name': 'Illinois', 'state_ID': 'il'},
 {'State Name': 'Indiana', 'state_ID': 'in'},
 {'State Name': 'Iowa', 'state_ID': 'ia'},
 {'State Name': 'Kansas', 'state_ID': 'ks'},
 {'State Name': 'Kentucky', 'state_ID': 'ky'},
 {'State Name': 'Louisiana', 'state_ID': 'la'},
 {'State Name': 'Maine', 'state_ID': 'me'},
 {'State Name': 'Maryland', 'state_ID': 'md'},
 {'State Name': 'Massachusetts', 'state_ID': 'ma'},
 {'State Name': 'Michigan', 'state_ID': 'mi'},
 {'Stat

In [16]:
state_names=pd.DataFrame(state_list)
state_names
state_names.to_csv("states.csv")

In [17]:
state_names

,State Name,state_ID
0,Alabama,al
1,Arizona,az
2,Arkansas,ar
3,California,ca
4,Colorado,co
5,Connecticut,ct
6,Delaware,de
7,District of Columbia,dc
8,Florida,fl
9,Georgia,ga


In [207]:
state_links

['al',
 'az',
 'ar',
 'ca',
 'co',
 'ct',
 'de',
 'dc',
 'fl',
 'ga',
 'id',
 'il',
 'in',
 'ia',
 'ks',
 'ky',
 'la',
 'me',
 'md',
 'ma',
 'mi',
 'mn',
 'ms',
 'mo',
 'mt',
 'ne',
 'nv',
 'nh',
 'nj',
 'nm',
 'ny',
 'nc',
 'nd',
 'oh',
 'ok',
 'or',
 'pa',
 'ri',
 'sc',
 'sd',
 'tn',
 'tx',
 'ut',
 'vt',
 '',
 'va',
 'wa',
 'wv',
 'wi',
 'wy']

In [18]:
state_links.remove('')


In [19]:
state_links

['al',
 'az',
 'ar',
 'ca',
 'co',
 'ct',
 'de',
 'dc',
 'fl',
 'ga',
 'id',
 'il',
 'in',
 'ia',
 'ks',
 'ky',
 'la',
 'me',
 'md',
 'ma',
 'mi',
 'mn',
 'ms',
 'mo',
 'mt',
 'ne',
 'nv',
 'nh',
 'nj',
 'nm',
 'ny',
 'nc',
 'nd',
 'oh',
 'ok',
 'or',
 'pa',
 'ri',
 'sc',
 'sd',
 'tn',
 'tx',
 'ut',
 'vt',
 'va',
 'wa',
 'wv',
 'wi',
 'wy']

In [213]:
base_url="https://www.dollargeneral.com/store-directory/"

# Part 2
The next step is to scrape each state-level directory for a list of towns in each state. 

In [20]:
city_state=[]
for state in state_links:
    response=requests.get(base_url+state)
    print(f"now scraping {state} response code is {response.status_code}")
    soup=BeautifulSoup(response.text,"html.parser")
    towns=soup.find_all("div", class_="city-list-item")
    for town in towns:
        town_name = town.a.get('title')
        my_dict={"city":town_name,
             "state":state}
        print(f"now adding{my_dict}")
        city_state.append(my_dict)
    snoozer()
town_names


now scraping al response code is 200
now adding{'city': 'Abbeville', 'state': 'al'}
now adding{'city': 'Addison', 'state': 'al'}
now adding{'city': 'Adger', 'state': 'al'}
now adding{'city': 'Akron', 'state': 'al'}
now adding{'city': 'Alabaster', 'state': 'al'}
now adding{'city': 'Albertville', 'state': 'al'}
now adding{'city': 'Alexander City', 'state': 'al'}
now adding{'city': 'Alexandria', 'state': 'al'}
now adding{'city': 'Aliceville', 'state': 'al'}
now adding{'city': 'Altoona', 'state': 'al'}
now adding{'city': 'Andalusia', 'state': 'al'}
now adding{'city': 'Anderson', 'state': 'al'}
now adding{'city': 'Anniston', 'state': 'al'}
now adding{'city': 'Arab', 'state': 'al'}
now adding{'city': 'Ardmore', 'state': 'al'}
now adding{'city': 'Ariton', 'state': 'al'}
now adding{'city': 'Arley', 'state': 'al'}
now adding{'city': 'Ashford', 'state': 'al'}
now adding{'city': 'Ashland', 'state': 'al'}
now adding{'city': 'Ashville', 'state': 'al'}
now adding{'city': 'Athens', 'state': 'al'}
now

now scraping az response code is 200
now adding{'city': 'Ajo', 'state': 'az'}
now adding{'city': 'Amado', 'state': 'az'}
now adding{'city': 'Apache Junction', 'state': 'az'}
now adding{'city': 'Arizona City', 'state': 'az'}
now adding{'city': 'Benson', 'state': 'az'}
now adding{'city': 'Bisbee', 'state': 'az'}
now adding{'city': 'Buckeye', 'state': 'az'}
now adding{'city': 'Bullhead City', 'state': 'az'}
now adding{'city': 'Camp Verde', 'state': 'az'}
now adding{'city': 'Casa Grande', 'state': 'az'}
now adding{'city': 'Chandler', 'state': 'az'}
now adding{'city': 'Chino Valley', 'state': 'az'}
now adding{'city': 'Clarkdale', 'state': 'az'}
now adding{'city': 'Clifton', 'state': 'az'}
now adding{'city': 'Colorado City', 'state': 'az'}
now adding{'city': 'Coolidge', 'state': 'az'}
now adding{'city': 'Cordes Lakes', 'state': 'az'}
now adding{'city': 'Cottonwood', 'state': 'az'}
now adding{'city': 'Dolan Springs', 'state': 'az'}
now adding{'city': 'Douglas', 'state': 'az'}
now adding{'city

now scraping ca response code is 200
now adding{'city': 'Adelanto', 'state': 'ca'}
now adding{'city': 'Alturas', 'state': 'ca'}
now adding{'city': 'Angels Camp', 'state': 'ca'}
now adding{'city': 'Antioch', 'state': 'ca'}
now adding{'city': 'Anza', 'state': 'ca'}
now adding{'city': 'Apple Valley', 'state': 'ca'}
now adding{'city': 'Arbuckle', 'state': 'ca'}
now adding{'city': 'Armona', 'state': 'ca'}
now adding{'city': 'Arnold', 'state': 'ca'}
now adding{'city': 'Arvin', 'state': 'ca'}
now adding{'city': 'Avenal', 'state': 'ca'}
now adding{'city': 'Bakersfield', 'state': 'ca'}
now adding{'city': 'Banning', 'state': 'ca'}
now adding{'city': 'Barstow', 'state': 'ca'}
now adding{'city': 'Blythe', 'state': 'ca'}
now adding{'city': 'Brawley', 'state': 'ca'}
now adding{'city': 'Brentwood', 'state': 'ca'}
now adding{'city': 'Brownsville', 'state': 'ca'}
now adding{'city': 'Burney', 'state': 'ca'}
now adding{'city': 'Cabazon', 'state': 'ca'}
now adding{'city': 'Calexico', 'state': 'ca'}
now ad

now scraping co response code is 200
now adding{'city': 'Alamosa', 'state': 'co'}
now adding{'city': 'Arvada', 'state': 'co'}
now adding{'city': 'Bailey', 'state': 'co'}
now adding{'city': 'Bayfield', 'state': 'co'}
now adding{'city': 'Bennett', 'state': 'co'}
now adding{'city': 'Brush', 'state': 'co'}
now adding{'city': 'Burlington', 'state': 'co'}
now adding{'city': 'Calhan', 'state': 'co'}
now adding{'city': 'Cedaredge', 'state': 'co'}
now adding{'city': 'Center', 'state': 'co'}
now adding{'city': 'Cheyenne Wells', 'state': 'co'}
now adding{'city': 'Colorado Springs', 'state': 'co'}
now adding{'city': 'Commerce City', 'state': 'co'}
now adding{'city': 'Cortez', 'state': 'co'}
now adding{'city': 'Dove Creek', 'state': 'co'}
now adding{'city': 'Eaton', 'state': 'co'}
now adding{'city': 'Estes Park', 'state': 'co'}
now adding{'city': 'Evans', 'state': 'co'}
now adding{'city': 'Fairplay', 'state': 'co'}
now adding{'city': 'Federal Heights', 'state': 'co'}
now adding{'city': 'Firestone',

now scraping ga response code is 200
now adding{'city': 'Abbeville', 'state': 'ga'}
now adding{'city': 'Acworth', 'state': 'ga'}
now adding{'city': 'Adairsville', 'state': 'ga'}
now adding{'city': 'Adel', 'state': 'ga'}
now adding{'city': 'Adrian', 'state': 'ga'}
now adding{'city': 'Alamo', 'state': 'ga'}
now adding{'city': 'Alapaha', 'state': 'ga'}
now adding{'city': 'Albany', 'state': 'ga'}
now adding{'city': 'Allentown', 'state': 'ga'}
now adding{'city': 'Alma', 'state': 'ga'}
now adding{'city': 'Alto', 'state': 'ga'}
now adding{'city': 'Ambrose', 'state': 'ga'}
now adding{'city': 'Americus', 'state': 'ga'}
now adding{'city': 'Appling', 'state': 'ga'}
now adding{'city': 'Arabi', 'state': 'ga'}
now adding{'city': 'Aragon', 'state': 'ga'}
now adding{'city': 'Arlington', 'state': 'ga'}
now adding{'city': 'Armuchee', 'state': 'ga'}
now adding{'city': 'Ashburn', 'state': 'ga'}
now adding{'city': 'Athens', 'state': 'ga'}
now adding{'city': 'Atlanta', 'state': 'ga'}
now adding{'city': 'Att

...

now scraping ut response code is 200
now adding{'city': 'Delta', 'state': 'ut'}
now adding{'city': 'Midvale', 'state': 'ut'}
now adding{'city': 'Moab', 'state': 'ut'}
now adding{'city': 'Nephi', 'state': 'ut'}
now adding{'city': 'Ogden', 'state': 'ut'}
now adding{'city': 'Price', 'state': 'ut'}
now adding{'city': 'Provo', 'state': 'ut'}
now adding{'city': 'Saint George', 'state': 'ut'}
now adding{'city': 'Salt Lake City', 'state': 'ut'}
now adding{'city': 'West Valley City', 'state': 'ut'}
now scraping vt response code is 200
now adding{'city': 'Alburgh', 'state': 'vt'}
now adding{'city': 'Arlington', 'state': 'vt'}
now adding{'city': 'Barre', 'state': 'vt'}
now adding{'city': 'Bennington', 'state': 'vt'}
now adding{'city': 'Berlin', 'state': 'vt'}
now adding{'city': 'Brattleboro', 'state': 'vt'}
now adding{'city': 'Castleton', 'state': 'vt'}
now adding{'city': 'Chester', 'state': 'vt'}
now adding{'city': 'Colchester', 'state': 'vt'}
now adding{'city': 'Enosburg Falls', 'state': 'vt'}


now scraping wa response code is 200
now adding{'city': 'Benton City', 'state': 'wa'}
now adding{'city': 'Bingen', 'state': 'wa'}
now adding{'city': 'Burbank', 'state': 'wa'}
now adding{'city': 'Carson', 'state': 'wa'}
now adding{'city': 'Castle Rock', 'state': 'wa'}
now adding{'city': 'Cathlamet', 'state': 'wa'}
now adding{'city': 'Centralia', 'state': 'wa'}
now adding{'city': 'Chattaroy', 'state': 'wa'}
now adding{'city': 'Chewelah', 'state': 'wa'}
now adding{'city': 'Connell', 'state': 'wa'}
now adding{'city': 'Elk', 'state': 'wa'}
now adding{'city': 'Goldendale', 'state': 'wa'}
now adding{'city': 'Grandview', 'state': 'wa'}
now adding{'city': 'Granger', 'state': 'wa'}
now adding{'city': 'Kittitas', 'state': 'wa'}
now adding{'city': 'Loon Lake', 'state': 'wa'}
now adding{'city': 'Mabton', 'state': 'wa'}
now adding{'city': 'Mattawa', 'state': 'wa'}
now adding{'city': 'Mccleary', 'state': 'wa'}
now adding{'city': 'Morton', 'state': 'wa'}
now adding{'city': 'Naches', 'state': 'wa'}
now

now scraping wi response code is 200
now adding{'city': 'Abbotsford', 'state': 'wi'}
now adding{'city': 'Adams', 'state': 'wi'}
now adding{'city': 'Albany', 'state': 'wi'}
now adding{'city': 'Amherst', 'state': 'wi'}
now adding{'city': 'Appleton', 'state': 'wi'}
now adding{'city': 'Arcadia', 'state': 'wi'}
now adding{'city': 'Argyle', 'state': 'wi'}
now adding{'city': 'Athens', 'state': 'wi'}
now adding{'city': 'Baldwin', 'state': 'wi'}
now adding{'city': 'Barron', 'state': 'wi'}
now adding{'city': 'Bay City', 'state': 'wi'}
now adding{'city': 'Beaver Dam', 'state': 'wi'}
now adding{'city': 'Beloit', 'state': 'wi'}
now adding{'city': 'Berlin', 'state': 'wi'}
now adding{'city': 'Big Flats', 'state': 'wi'}
now adding{'city': 'Birchwood', 'state': 'wi'}
now adding{'city': 'Birnamwood', 'state': 'wi'}
now adding{'city': 'Black Creek', 'state': 'wi'}
now adding{'city': 'Black River Falls', 'state': 'wi'}
now adding{'city': 'Blair', 'state': 'wi'}
now adding{'city': 'Bloomer', 'state': 'wi'}

now scraping wy response code is 200
now adding{'city': 'Casper', 'state': 'wy'}
now adding{'city': 'Cheyenne', 'state': 'wy'}
now adding{'city': 'Douglas', 'state': 'wy'}
now adding{'city': 'Gillette', 'state': 'wy'}
now adding{'city': 'Glenrock', 'state': 'wy'}
now adding{'city': 'Guernsey', 'state': 'wy'}
now adding{'city': 'Laramie', 'state': 'wy'}
now adding{'city': 'Moorcroft', 'state': 'wy'}
now adding{'city': 'Newcastle', 'state': 'wy'}
now adding{'city': 'Pine Bluffs', 'state': 'wy'}
now adding{'city': 'Wheatland', 'state': 'wy'}
now adding{'city': 'Wright', 'state': 'wy'}


NameError: name 'town_names' is not defined

In [21]:
city_df=pd.DataFrame(city_state)
city_df

,city,state
0,Abbeville,al
1,Addison,al
2,Adger,al
3,Akron,al
4,Alabaster,al
...,...,...
10889,Moorcroft,wy
10890,Newcastle,wy
10891,Pine Bluffs,wy
10892,Wheatland,wy


In [22]:
city_state

[{'city': 'Abbeville', 'state': 'al'},
 {'city': 'Addison', 'state': 'al'},
 {'city': 'Adger', 'state': 'al'},
 {'city': 'Akron', 'state': 'al'},
 {'city': 'Alabaster', 'state': 'al'},
 {'city': 'Albertville', 'state': 'al'},
 {'city': 'Alexander City', 'state': 'al'},
 {'city': 'Alexandria', 'state': 'al'},
 {'city': 'Aliceville', 'state': 'al'},
 {'city': 'Altoona', 'state': 'al'},
 {'city': 'Andalusia', 'state': 'al'},
 {'city': 'Anderson', 'state': 'al'},
 {'city': 'Anniston', 'state': 'al'},
 {'city': 'Arab', 'state': 'al'},
 {'city': 'Ardmore', 'state': 'al'},
 {'city': 'Ariton', 'state': 'al'},
 {'city': 'Arley', 'state': 'al'},
 {'city': 'Ashford', 'state': 'al'},
 {'city': 'Ashland', 'state': 'al'},
 {'city': 'Ashville', 'state': 'al'},
 {'city': 'Athens', 'state': 'al'},
 {'city': 'Atmore', 'state': 'al'},
 {'city': 'Attalla', 'state': 'al'},
 {'city': 'Auburn', 'state': 'al'},
 {'city': 'Autaugaville', 'state': 'al'},
 {'city': 'Baileyton', 'state': 'al'},
 {'city': 'Bay Min

In [23]:
city_df.to_csv("cities.csv")

# Part 3
The final step is to scrape each town-level directory to find the number of DG stores in that town. Each store is associated with a unique store_card item in html. 

In [24]:
base_url

'https://www.dollargeneral.com/store-directory/'

In [197]:
response=requests.get("https://www.dollargeneral.com/store-directory/al/birmingham")
response.status_code

200

In [198]:
soup=BeautifulSoup(response.text, "html.parser")
soup


<!DOCTYPE HTML>

<html lang="en">
<head>
<meta charset="utf-8"/>
<title>
        Birmingham | Dollar General
    </title>
<!-- <meta data-sly-test="false" name="robots" content="noindex,nofollow"/> -->
<meta content="store-locations" name="template"/>
<meta content="width=device-width, initial-scale=1" name="viewport"/>
<base href="https://www.dollargeneral.com"/>
<link href="https://fonts.googleapis.com/css2?family=Montserrat:ital,wght@0,100;0,200;0,300;0,400;0,500;0,600;0,700;0,800;0,900;1,100;1,200;1,300;1,400;1,500;1,600;1,700;1,800;1,900&amp;display=swap" rel="stylesheet"/>
<script src="https://polyfill.io/v3/polyfill.min.js?features=HTMLTemplateElement%2CString.prototype.includes%2CArray.from%2CNodeList.prototype.forEach%2CURLSearchParams%2CElement.prototype.append">
</script>
<meta content="www.dollargeneral.com" name="test-serverName"/>
<meta content="www.dollargeneral.com" name="test-domainName"/>
<link href="https://www.dollargeneral.com/store-directory/al/birmingham" rel="c

In [200]:
stores=soup.find_all("div", class_="store__card")
len(stores)

18

In [26]:
city_state_stores=[]
for place in city_state:
    state=place.get("state")
    town=place.get("city").lower().replace(' ','-')
    url=base_url+state+"/"+town
    response=requests.get(url)
    print(f"{url} is response {response.status_code}")
    soup=BeautifulSoup(response.text,"html.parser")
    stores=soup.find_all("div", class_="store__card")
    store_number=len(stores)
    my_dict={"city":town,
            "state":state,
            "stores":store_number}   
    city_state_stores.append(my_dict)
    snoozer()
city_state_stores

https://www.dollargeneral.com/store-directory/al/abbeville is response 200
https://www.dollargeneral.com/store-directory/al/addison is response 200
https://www.dollargeneral.com/store-directory/al/adger is response 200
https://www.dollargeneral.com/store-directory/al/akron is response 200
https://www.dollargeneral.com/store-directory/al/alabaster is response 200
https://www.dollargeneral.com/store-directory/al/albertville is response 200
https://www.dollargeneral.com/store-directory/al/alexander-city is response 200
https://www.dollargeneral.com/store-directory/al/alexandria is response 200
https://www.dollargeneral.com/store-directory/al/aliceville is response 200
https://www.dollargeneral.com/store-directory/al/altoona is response 200
https://www.dollargeneral.com/store-directory/al/andalusia is response 200
https://www.dollargeneral.com/store-directory/al/anderson is response 200
https://www.dollargeneral.com/store-directory/al/anniston is response 200
https://www.dollargeneral.com/

https://www.dollargeneral.com/store-directory/al/elmore is response 200
https://www.dollargeneral.com/store-directory/al/enterprise is response 200
https://www.dollargeneral.com/store-directory/al/eufaula is response 200
https://www.dollargeneral.com/store-directory/al/eutaw is response 200
https://www.dollargeneral.com/store-directory/al/eva is response 200
https://www.dollargeneral.com/store-directory/al/evergreen is response 200
https://www.dollargeneral.com/store-directory/al/excel is response 200
https://www.dollargeneral.com/store-directory/al/fairhope is response 200
https://www.dollargeneral.com/store-directory/al/falkville is response 200
https://www.dollargeneral.com/store-directory/al/fayette is response 200
https://www.dollargeneral.com/store-directory/al/five-points is response 200
https://www.dollargeneral.com/store-directory/al/flat-rock is response 200
https://www.dollargeneral.com/store-directory/al/flomaton is response 200
https://www.dollargeneral.com/store-directory

https://www.dollargeneral.com/store-directory/al/madison is response 200
https://www.dollargeneral.com/store-directory/al/magnolia-springs is response 200
https://www.dollargeneral.com/store-directory/al/malvern is response 200
https://www.dollargeneral.com/store-directory/al/maplesville is response 200
https://www.dollargeneral.com/store-directory/al/marion is response 200
https://www.dollargeneral.com/store-directory/al/mccalla is response 200
https://www.dollargeneral.com/store-directory/al/mcintosh is response 200
https://www.dollargeneral.com/store-directory/al/mckenzie is response 200
https://www.dollargeneral.com/store-directory/al/meridianville is response 200
https://www.dollargeneral.com/store-directory/al/midland-city is response 200
https://www.dollargeneral.com/store-directory/al/midway is response 200
https://www.dollargeneral.com/store-directory/al/millbrook is response 200
https://www.dollargeneral.com/store-directory/al/millport is response 200
https://www.dollargenera

https://www.dollargeneral.com/store-directory/al/sylacauga is response 200
https://www.dollargeneral.com/store-directory/al/sylvan-springs is response 200
https://www.dollargeneral.com/store-directory/al/sylvania is response 200
https://www.dollargeneral.com/store-directory/al/talladega is response 200
https://www.dollargeneral.com/store-directory/al/tallassee is response 200
https://www.dollargeneral.com/store-directory/al/tanner is response 200
https://www.dollargeneral.com/store-directory/al/theodore is response 200
https://www.dollargeneral.com/store-directory/al/thomaston is response 200
https://www.dollargeneral.com/store-directory/al/thomasville is response 200
https://www.dollargeneral.com/store-directory/al/thorsby is response 200
https://www.dollargeneral.com/store-directory/al/titus is response 200
https://www.dollargeneral.com/store-directory/al/toney is response 200
https://www.dollargeneral.com/store-directory/al/town-creek is response 200
https://www.dollargeneral.com/st

https://www.dollargeneral.com/store-directory/az/sun-city is response 200
https://www.dollargeneral.com/store-directory/az/sunsites is response 200
https://www.dollargeneral.com/store-directory/az/surprise is response 200
https://www.dollargeneral.com/store-directory/az/tonopah is response 200
https://www.dollargeneral.com/store-directory/az/tonto-basin is response 200
https://www.dollargeneral.com/store-directory/az/tuba-city is response 200
https://www.dollargeneral.com/store-directory/az/tucson is response 200
https://www.dollargeneral.com/store-directory/az/wellton is response 200
https://www.dollargeneral.com/store-directory/az/wickenburg is response 200
https://www.dollargeneral.com/store-directory/az/willcox is response 200
https://www.dollargeneral.com/store-directory/az/williams is response 200
https://www.dollargeneral.com/store-directory/az/winslow is response 200
https://www.dollargeneral.com/store-directory/az/wittmann is response 200
https://www.dollargeneral.com/store-di

https://www.dollargeneral.com/store-directory/ar/garfield is response 200
https://www.dollargeneral.com/store-directory/ar/gassville is response 200
https://www.dollargeneral.com/store-directory/ar/gentry is response 200
https://www.dollargeneral.com/store-directory/ar/gillett is response 200
https://www.dollargeneral.com/store-directory/ar/glenwood is response 200
https://www.dollargeneral.com/store-directory/ar/gosnell is response 200
https://www.dollargeneral.com/store-directory/ar/gould is response 200
https://www.dollargeneral.com/store-directory/ar/gravette is response 200
https://www.dollargeneral.com/store-directory/ar/green-forest is response 200
https://www.dollargeneral.com/store-directory/ar/greenbrier is response 200
https://www.dollargeneral.com/store-directory/ar/greenwood is response 200
https://www.dollargeneral.com/store-directory/ar/greers-ferry is response 200
https://www.dollargeneral.com/store-directory/ar/gurdon is response 200
https://www.dollargeneral.com/store

https://www.dollargeneral.com/store-directory/ar/newport is response 200
https://www.dollargeneral.com/store-directory/ar/norman is response 200
https://www.dollargeneral.com/store-directory/ar/north-little-rock is response 200
https://www.dollargeneral.com/store-directory/ar/ola is response 200
https://www.dollargeneral.com/store-directory/ar/omaha is response 200
https://www.dollargeneral.com/store-directory/ar/oppelo is response 200
https://www.dollargeneral.com/store-directory/ar/osceola is response 200
https://www.dollargeneral.com/store-directory/ar/oxford is response 200
https://www.dollargeneral.com/store-directory/ar/ozark is response 200
https://www.dollargeneral.com/store-directory/ar/palestine is response 200
https://www.dollargeneral.com/store-directory/ar/pangburn is response 200
https://www.dollargeneral.com/store-directory/ar/paragould is response 200
https://www.dollargeneral.com/store-directory/ar/paris is response 200
https://www.dollargeneral.com/store-directory/ar/

KeyboardInterrupt: 

In [29]:
len(city_state_stores)

702

In [32]:
length=len(city_state_stores)
broken_links=["https://www.dollargeneral.com/store-directory/ar/huntington"]
for place in city_state[length:]:
    state=place.get("state")
    town=place.get("city").lower().replace(' ','-')
    url=base_url+state+"/"+town
    response=requests.get(url)
    print(f"{url} is response {response.status_code}")
    soup=BeautifulSoup(response.text,"html.parser")
    stores=soup.find_all("div", class_="store__card")
    store_number=len(stores)
    my_dict={"city":town,
            "state":state,
            "stores":store_number}   
    city_state_stores.append(my_dict)
    if response.status_code>=400:
        broken_links.append(url)
    snoozer()

https://www.dollargeneral.com/store-directory/ar/rison is response 200
https://www.dollargeneral.com/store-directory/ar/rogers is response 200
https://www.dollargeneral.com/store-directory/ar/roland is response 200
https://www.dollargeneral.com/store-directory/ar/romance is response 200
https://www.dollargeneral.com/store-directory/ar/rose-bud is response 200
https://www.dollargeneral.com/store-directory/ar/rosston is response 200
https://www.dollargeneral.com/store-directory/ar/royal is response 200
https://www.dollargeneral.com/store-directory/ar/rudy is response 200
https://www.dollargeneral.com/store-directory/ar/russellville is response 200
https://www.dollargeneral.com/store-directory/ar/saint-joe is response 200
https://www.dollargeneral.com/store-directory/ar/saint-paul is response 200
https://www.dollargeneral.com/store-directory/ar/salem is response 200
https://www.dollargeneral.com/store-directory/ar/salesville is response 200
https://www.dollargeneral.com/store-directory/ar

https://www.dollargeneral.com/store-directory/ca/etna is response 200
https://www.dollargeneral.com/store-directory/ca/exeter is response 200
https://www.dollargeneral.com/store-directory/ca/fall-river-mills is response 200
https://www.dollargeneral.com/store-directory/ca/farmersville is response 200
https://www.dollargeneral.com/store-directory/ca/fillmore is response 200
https://www.dollargeneral.com/store-directory/ca/firebaugh is response 200
https://www.dollargeneral.com/store-directory/ca/foresthill is response 200
https://www.dollargeneral.com/store-directory/ca/fortuna is response 200
https://www.dollargeneral.com/store-directory/ca/fowler is response 200
https://www.dollargeneral.com/store-directory/ca/frazier-park is response 200
https://www.dollargeneral.com/store-directory/ca/fresno is response 200
https://www.dollargeneral.com/store-directory/ca/goshen is response 200
https://www.dollargeneral.com/store-directory/ca/grass-valley is response 200
https://www.dollargeneral.co

https://www.dollargeneral.com/store-directory/ca/somerset is response 200
https://www.dollargeneral.com/store-directory/ca/sonora is response 200
https://www.dollargeneral.com/store-directory/ca/squaw-valley is response 200
https://www.dollargeneral.com/store-directory/ca/stockton is response 200
https://www.dollargeneral.com/store-directory/ca/strathmore is response 200
https://www.dollargeneral.com/store-directory/ca/sun-village is response 200
https://www.dollargeneral.com/store-directory/ca/susanville is response 200
https://www.dollargeneral.com/store-directory/ca/sutter is response 200
https://www.dollargeneral.com/store-directory/ca/taft is response 200
https://www.dollargeneral.com/store-directory/ca/tehachapi is response 200
https://www.dollargeneral.com/store-directory/ca/tipton is response 200
https://www.dollargeneral.com/store-directory/ca/tulare is response 200
https://www.dollargeneral.com/store-directory/ca/turlock is response 200
https://www.dollargeneral.com/store-dir

https://www.dollargeneral.com/store-directory/ct/colchester is response 200
https://www.dollargeneral.com/store-directory/ct/coventry is response 200
https://www.dollargeneral.com/store-directory/ct/cromwell is response 200
https://www.dollargeneral.com/store-directory/ct/dayville is response 200
https://www.dollargeneral.com/store-directory/ct/east-hampton is response 200
https://www.dollargeneral.com/store-directory/ct/east-hartford is response 200
https://www.dollargeneral.com/store-directory/ct/east-haven is response 200
https://www.dollargeneral.com/store-directory/ct/east-lyme is response 200
https://www.dollargeneral.com/store-directory/ct/eastford is response 200
https://www.dollargeneral.com/store-directory/ct/enfield is response 200
https://www.dollargeneral.com/store-directory/ct/gales-ferry is response 200
https://www.dollargeneral.com/store-directory/ct/griswold is response 200
https://www.dollargeneral.com/store-directory/ct/groton is response 200
https://www.dollargenera

https://www.dollargeneral.com/store-directory/fl/bonifay is response 200
https://www.dollargeneral.com/store-directory/fl/bonita-springs is response 200
https://www.dollargeneral.com/store-directory/fl/bowling-green is response 200
https://www.dollargeneral.com/store-directory/fl/boynton-beach is response 200
https://www.dollargeneral.com/store-directory/fl/bradenton is response 200
https://www.dollargeneral.com/store-directory/fl/brandon is response 200
https://www.dollargeneral.com/store-directory/fl/branford is response 200
https://www.dollargeneral.com/store-directory/fl/bristol is response 200
https://www.dollargeneral.com/store-directory/fl/bronson is response 200
https://www.dollargeneral.com/store-directory/fl/brooker is response 200
https://www.dollargeneral.com/store-directory/fl/brooksville is response 200
https://www.dollargeneral.com/store-directory/fl/bryceville is response 200
https://www.dollargeneral.com/store-directory/fl/bunnell is response 200
https://www.dollargene

...

https://www.dollargeneral.com/store-directory/fl/old-town is response 200
https://www.dollargeneral.com/store-directory/fl/opa-locka is response 200
https://www.dollargeneral.com/store-directory/fl/orange-city is response 200
https://www.dollargeneral.com/store-directory/fl/orange-park is response 200
https://www.dollargeneral.com/store-directory/fl/orange-springs is response 200
https://www.dollargeneral.com/store-directory/fl/orlando is response 200
https://www.dollargeneral.com/store-directory/fl/ormond-beach is response 200
https://www.dollargeneral.com/store-directory/fl/osteen is response 200
https://www.dollargeneral.com/store-directory/fl/oviedo is response 200
https://www.dollargeneral.com/store-directory/fl/pace is response 200
https://www.dollargeneral.com/store-directory/fl/pahokee is response 200
https://www.dollargeneral.com/store-directory/fl/paisley is response 200
https://www.dollargeneral.com/store-directory/fl/palatka is response 200
https://www.dollargeneral.com/sto

https://www.dollargeneral.com/store-directory/fl/wimauma is response 200
https://www.dollargeneral.com/store-directory/fl/winter-garden is response 200
https://www.dollargeneral.com/store-directory/fl/winter-haven is response 200
https://www.dollargeneral.com/store-directory/fl/winter-park is response 200
https://www.dollargeneral.com/store-directory/fl/winter-springs is response 200
https://www.dollargeneral.com/store-directory/fl/youngstown is response 200
https://www.dollargeneral.com/store-directory/fl/yulee is response 200
https://www.dollargeneral.com/store-directory/fl/zellwood is response 200
https://www.dollargeneral.com/store-directory/fl/zephyrhills is response 200
https://www.dollargeneral.com/store-directory/fl/zolfo-springs is response 200
https://www.dollargeneral.com/store-directory/ga/abbeville is response 200
https://www.dollargeneral.com/store-directory/ga/acworth is response 200
https://www.dollargeneral.com/store-directory/ga/adairsville is response 200
https://www

https://www.dollargeneral.com/store-directory/ga/concord is response 200
https://www.dollargeneral.com/store-directory/ga/conyers is response 200
https://www.dollargeneral.com/store-directory/ga/coolidge is response 200
https://www.dollargeneral.com/store-directory/ga/coosa is response 200
https://www.dollargeneral.com/store-directory/ga/cordele is response 200
https://www.dollargeneral.com/store-directory/ga/cornelia is response 200
https://www.dollargeneral.com/store-directory/ga/covington is response 200
https://www.dollargeneral.com/store-directory/ga/crandall is response 200
https://www.dollargeneral.com/store-directory/ga/crawford is response 200
https://www.dollargeneral.com/store-directory/ga/crawfordville is response 200
https://www.dollargeneral.com/store-directory/ga/cumming is response 200
https://www.dollargeneral.com/store-directory/ga/cusseta is response 200
https://www.dollargeneral.com/store-directory/ga/cuthbert is response 200
https://www.dollargeneral.com/store-dire

https://www.dollargeneral.com/store-directory/ga/hoschton is response 200
https://www.dollargeneral.com/store-directory/ga/hull is response 200
https://www.dollargeneral.com/store-directory/ga/ideal is response 200
https://www.dollargeneral.com/store-directory/ga/ila is response 200
https://www.dollargeneral.com/store-directory/ga/iron-city is response 200
https://www.dollargeneral.com/store-directory/ga/irwinton is response 200
https://www.dollargeneral.com/store-directory/ga/jackson is response 200
https://www.dollargeneral.com/store-directory/ga/jacksonville is response 200
https://www.dollargeneral.com/store-directory/ga/jakin is response 200
https://www.dollargeneral.com/store-directory/ga/jasper is response 200
https://www.dollargeneral.com/store-directory/ga/jefferson is response 200
https://www.dollargeneral.com/store-directory/ga/jeffersonville is response 200
https://www.dollargeneral.com/store-directory/ga/jenkinsburg is response 200
https://www.dollargeneral.com/store-direc

KeyboardInterrupt: 

In [34]:
length=len(city_state_stores)
for place in city_state[length:]:
    state=place.get("state")
    town=place.get("city").lower().replace(' ','-')
    url=base_url+state+"/"+town
    response=requests.get(url)
    print(f"{url} is response {response.status_code}")
    soup=BeautifulSoup(response.text,"html.parser")
    stores=soup.find_all("div", class_="store__card")
    store_number=len(stores)
    my_dict={"city":town,
            "state":state,
            "stores":store_number}   
    city_state_stores.append(my_dict)
    if response.status_code>=400:
        broken_links.append(url)
    snoozer()

https://www.dollargeneral.com/store-directory/ga/mount-airy is response 200
https://www.dollargeneral.com/store-directory/ga/mount-vernon is response 200
https://www.dollargeneral.com/store-directory/ga/murrayville is response 200
https://www.dollargeneral.com/store-directory/ga/nahunta is response 200
https://www.dollargeneral.com/store-directory/ga/nashville is response 200
https://www.dollargeneral.com/store-directory/ga/naylor is response 200
https://www.dollargeneral.com/store-directory/ga/newborn is response 200
https://www.dollargeneral.com/store-directory/ga/newington is response 200
https://www.dollargeneral.com/store-directory/ga/newnan is response 200
https://www.dollargeneral.com/store-directory/ga/newton is response 200
https://www.dollargeneral.com/store-directory/ga/nicholls is response 200
https://www.dollargeneral.com/store-directory/ga/nicholson is response 200
https://www.dollargeneral.com/store-directory/ga/norcross is response 200
https://www.dollargeneral.com/stor

https://www.dollargeneral.com/store-directory/ga/tifton is response 200
https://www.dollargeneral.com/store-directory/ga/tiger is response 200
https://www.dollargeneral.com/store-directory/ga/tignall is response 200
https://www.dollargeneral.com/store-directory/ga/toccoa is response 200
https://www.dollargeneral.com/store-directory/ga/toomsboro is response 200
https://www.dollargeneral.com/store-directory/ga/townsend is response 200
https://www.dollargeneral.com/store-directory/ga/trenton is response 200
https://www.dollargeneral.com/store-directory/ga/trion is response 200
https://www.dollargeneral.com/store-directory/ga/tucker is response 200
https://www.dollargeneral.com/store-directory/ga/tunnel-hill is response 200
https://www.dollargeneral.com/store-directory/ga/twin-city is response 200
https://www.dollargeneral.com/store-directory/ga/ty-ty is response 200
https://www.dollargeneral.com/store-directory/ga/tyrone is response 200
https://www.dollargeneral.com/store-directory/ga/una

https://www.dollargeneral.com/store-directory/il/bridgeport is response 200
https://www.dollargeneral.com/store-directory/il/brighton is response 200
https://www.dollargeneral.com/store-directory/il/brimfield is response 200
https://www.dollargeneral.com/store-directory/il/brookport is response 404
https://www.dollargeneral.com/store-directory/il/brownstown is response 200
https://www.dollargeneral.com/store-directory/il/buffalo is response 200
https://www.dollargeneral.com/store-directory/il/bunker-hill is response 200
https://www.dollargeneral.com/store-directory/il/burbank is response 200
https://www.dollargeneral.com/store-directory/il/bushnell is response 200
https://www.dollargeneral.com/store-directory/il/byron is response 200
https://www.dollargeneral.com/store-directory/il/cahokia is response 200
https://www.dollargeneral.com/store-directory/il/cairo is response 200
https://www.dollargeneral.com/store-directory/il/calumet-city is response 200
https://www.dollargeneral.com/stor

https://www.dollargeneral.com/store-directory/il/farina is response 200
https://www.dollargeneral.com/store-directory/il/farmer-city is response 200
https://www.dollargeneral.com/store-directory/il/farmington is response 200
https://www.dollargeneral.com/store-directory/il/findlay is response 200
https://www.dollargeneral.com/store-directory/il/fisher is response 200
https://www.dollargeneral.com/store-directory/il/flanagan is response 200
https://www.dollargeneral.com/store-directory/il/flora is response 200
https://www.dollargeneral.com/store-directory/il/forreston is response 200
https://www.dollargeneral.com/store-directory/il/fox-lake is response 200
https://www.dollargeneral.com/store-directory/il/freeburg is response 200
https://www.dollargeneral.com/store-directory/il/freeport is response 200
https://www.dollargeneral.com/store-directory/il/fulton is response 200
https://www.dollargeneral.com/store-directory/il/galatia is response 200
https://www.dollargeneral.com/store-directo

https://www.dollargeneral.com/store-directory/il/mahomet is response 200
https://www.dollargeneral.com/store-directory/il/manhattan is response 200
https://www.dollargeneral.com/store-directory/il/manito is response 200
https://www.dollargeneral.com/store-directory/il/manteno is response 200
https://www.dollargeneral.com/store-directory/il/marengo is response 200
https://www.dollargeneral.com/store-directory/il/marion is response 200
https://www.dollargeneral.com/store-directory/il/marissa is response 200
https://www.dollargeneral.com/store-directory/il/maroa is response 200
https://www.dollargeneral.com/store-directory/il/marquette-heights is response 200
https://www.dollargeneral.com/store-directory/il/marseilles is response 200
https://www.dollargeneral.com/store-directory/il/marshall is response 200
https://www.dollargeneral.com/store-directory/il/martinsville is response 200
https://www.dollargeneral.com/store-directory/il/maryville is response 200
https://www.dollargeneral.com/st

...

https://www.dollargeneral.com/store-directory/wa/morton is response 200
https://www.dollargeneral.com/store-directory/wa/naches is response 200
https://www.dollargeneral.com/store-directory/wa/napavine is response 200
https://www.dollargeneral.com/store-directory/wa/oakville is response 200
https://www.dollargeneral.com/store-directory/wa/ocean-park is response 200
https://www.dollargeneral.com/store-directory/wa/ocean-shores is response 200
https://www.dollargeneral.com/store-directory/wa/pasco is response 200
https://www.dollargeneral.com/store-directory/wa/pe-ell is response 200
https://www.dollargeneral.com/store-directory/wa/pomeroy is response 200
https://www.dollargeneral.com/store-directory/wa/rainier is response 404
https://www.dollargeneral.com/store-directory/wa/soap-lake is response 200
https://www.dollargeneral.com/store-directory/wa/south-bend is response 200
https://www.dollargeneral.com/store-directory/wa/springdale is response 404
https://www.dollargeneral.com/store-di

https://www.dollargeneral.com/store-directory/wv/kermit is response 200
https://www.dollargeneral.com/store-directory/wv/keyser is response 200
https://www.dollargeneral.com/store-directory/wv/kingwood is response 200
https://www.dollargeneral.com/store-directory/wv/lavalette is response 200
https://www.dollargeneral.com/store-directory/wv/leon is response 200
https://www.dollargeneral.com/store-directory/wv/lerona is response 200
https://www.dollargeneral.com/store-directory/wv/lesage is response 200
https://www.dollargeneral.com/store-directory/wv/lewisburg is response 200
https://www.dollargeneral.com/store-directory/wv/lindside is response 200
https://www.dollargeneral.com/store-directory/wv/logan is response 200
https://www.dollargeneral.com/store-directory/wv/lookout is response 200
https://www.dollargeneral.com/store-directory/wv/madison is response 200
https://www.dollargeneral.com/store-directory/wv/man is response 200
https://www.dollargeneral.com/store-directory/wv/manningto

https://www.dollargeneral.com/store-directory/wv/welch is response 200
https://www.dollargeneral.com/store-directory/wv/wellsburg is response 200
https://www.dollargeneral.com/store-directory/wv/west-milford is response 200
https://www.dollargeneral.com/store-directory/wv/west-union is response 200
https://www.dollargeneral.com/store-directory/wv/weston is response 200
https://www.dollargeneral.com/store-directory/wv/wheeling is response 200
https://www.dollargeneral.com/store-directory/wv/white-hall is response 200
https://www.dollargeneral.com/store-directory/wv/whitesville is response 200
https://www.dollargeneral.com/store-directory/wv/wilcoe is response 200
https://www.dollargeneral.com/store-directory/wv/williamson is response 200
https://www.dollargeneral.com/store-directory/wv/williamstown is response 200
https://www.dollargeneral.com/store-directory/wv/winfield is response 200
https://www.dollargeneral.com/store-directory/wv/worthington is response 200
https://www.dollargenera

https://www.dollargeneral.com/store-directory/wi/kenosha is response 200
https://www.dollargeneral.com/store-directory/wi/kewaunee is response 200
https://www.dollargeneral.com/store-directory/wi/kiel is response 200
https://www.dollargeneral.com/store-directory/wi/kimberly is response 200
https://www.dollargeneral.com/store-directory/wi/kronenwetter is response 200
https://www.dollargeneral.com/store-directory/wi/la-farge is response 200
https://www.dollargeneral.com/store-directory/wi/ladysmith is response 200
https://www.dollargeneral.com/store-directory/wi/lake-geneva is response 200
https://www.dollargeneral.com/store-directory/wi/lake-mills is response 200
https://www.dollargeneral.com/store-directory/wi/lake-wazeecha is response 200
https://www.dollargeneral.com/store-directory/wi/lake-wissota is response 200
https://www.dollargeneral.com/store-directory/wi/lakewood is response 200
https://www.dollargeneral.com/store-directory/wi/lancaster is response 200
https://www.dollargener

https://www.dollargeneral.com/store-directory/wi/stevens-point is response 200
https://www.dollargeneral.com/store-directory/wi/stone-lake is response 200
https://www.dollargeneral.com/store-directory/wi/stoughton is response 200
https://www.dollargeneral.com/store-directory/wi/stratford is response 200
https://www.dollargeneral.com/store-directory/wi/sun-prairie is response 200
https://www.dollargeneral.com/store-directory/wi/superior is response 200
https://www.dollargeneral.com/store-directory/wi/suring is response 200
https://www.dollargeneral.com/store-directory/wi/theresa is response 200
https://www.dollargeneral.com/store-directory/wi/three-lakes is response 200
https://www.dollargeneral.com/store-directory/wi/tomahawk is response 200
https://www.dollargeneral.com/store-directory/wi/trevor is response 200
https://www.dollargeneral.com/store-directory/wi/turtle-lake is response 200
https://www.dollargeneral.com/store-directory/wi/twin-lakes is response 200
https://www.dollargener

In [35]:
city_state_stores

[{'city': 'abbeville', 'state': 'al', 'stores': 1},
 {'city': 'addison', 'state': 'al', 'stores': 1},
 {'city': 'adger', 'state': 'al', 'stores': 1},
 {'city': 'akron', 'state': 'al', 'stores': 1},
 {'city': 'alabaster', 'state': 'al', 'stores': 2},
 {'city': 'albertville', 'state': 'al', 'stores': 6},
 {'city': 'alexander-city', 'state': 'al', 'stores': 5},
 {'city': 'alexandria', 'state': 'al', 'stores': 1},
 {'city': 'aliceville', 'state': 'al', 'stores': 1},
 {'city': 'altoona', 'state': 'al', 'stores': 2},
 {'city': 'andalusia', 'state': 'al', 'stores': 5},
 {'city': 'anderson', 'state': 'al', 'stores': 1},
 {'city': 'anniston', 'state': 'al', 'stores': 7},
 {'city': 'arab', 'state': 'al', 'stores': 2},
 {'city': 'ardmore', 'state': 'al', 'stores': 1},
 {'city': 'ariton', 'state': 'al', 'stores': 1},
 {'city': 'arley', 'state': 'al', 'stores': 1},
 {'city': 'ashford', 'state': 'al', 'stores': 1},
 {'city': 'ashland', 'state': 'al', 'stores': 2},
 {'city': 'ashville', 'state': 'al'

In [36]:
df=pd.DataFrame(city_state_stores)
df

,city,state,stores
0,abbeville,al,1
1,addison,al,1
2,adger,al,1
3,akron,al,1
4,alabaster,al,2
...,...,...,...
10889,moorcroft,wy,1
10890,newcastle,wy,1
10891,pine-bluffs,wy,1
10892,wheatland,wy,1


In [38]:
df.to_csv("DG towns and stores.csv")

The final step of this analysis is to map each county to the numerical county code. The easiest way to do this is in Excel or Google Sheets, using VLOOKUP. I then mapped those figures in Datawrapper.